In [457]:
import csv
import glob
import os
import pydicom
from datetime import date, time, datetime
import shutil

deidentification_tag = [
    0x00080020, # StudyDate
    0x00080021, # SeriesDate
    0x00080022, # AcquisitionDate
    0x00080023, # ContentDate
    0x00080024, # OverlayDate
    0x00080025, # CurveDate
    0x0008002A, # AcquisitionDatetime
    0x00080030, # StudyTime
    0x00080031, # SeriesTime
    0x00080032, # AcquisitionTime
    0x00080033, # ContentTime
    0x00080034, # OverlayTime
    0x00080035, # CurveTime
    0x00080050, # AccessionNumber
    0x00080080, # InstitutionName
    0x00080081, # InstiutionAddress
    0x00080090, # ReferringPhysiciansName
    0x00080092, # ReferringPhysiciansTelephonNumber
    0x00080094, # ReferringPhysiciansAddress
    0x00080096, # ReferringPhysiciansIDSequence
    0x00081040, # InstitutionalDepartmentName
    0x00081048, # PhysiciansOfRecord
    0x00081049, # PhysicianOfRecordIDSequence
    0x00081050, # PerformingPhysiciansName
    0x00081052, # PerformingPhysiciansIDSequence
    0x00081060, # NameOfPhysicianReadingStudy
    0x00081062, # PhysicianReadingStudyIDSequence
    0x00081070, # OperatorsName
    0x00100010, # PatientsName
    0x00100020, # PatientID
    0x00100021, # IssuerOfPatientID
    0x00100030, # PatientsBirthDate
    0x00100032, # PatientsBirthTime
    0x00100040, # PatientsSex
    0x00101000, # OtherPatientIDs
    0x00101001, # OtherPatientNames
    0x00101005, # PatientsBirthName
    0x00101010, # PatientAge
    0x00101040, # PatientsAddress
    0x00101060, # PatientsMothersBirthName
    0x00102150, # CountryOfResidence
    0x00102152, # RegionOfResidence
    0x00102154, # PatientsTelephoneNumbers
    0x00200010, # StudyID
    0x00380300, # CurrentPatientLocation
    0x00380400, # PatientsInstitutionResidence
    0x0400A120, # DateTime
    0x0040A121, # Date
    0x0040A122, # Time
    0x0040A123  # PersonName
]

deidentification_name = [
    'StudyDate',
    'SeriesDate',
    'AcquisitionDate',
    'ContentDate',
    'OverlayDate',
    'CurveDate',
    'AcquisitionDatetime',
    'StudyTime',
    'SeriesTime',
    'AcquisitionTime',
    'ContentTime',
    'OverlayTime',
    'CurveTime',
    'AccessionNumber',
    'InstitutionName',
    'InstiutionAddress',
    'ReferringPhysiciansName',
    'ReferringPhysiciansTelephonNumber',
    'ReferringPhysiciansAddress',
    'ReferringPhysiciansIDSequence',
    'InstitutionalDepartmentName',
    'PhysiciansOfRecord',
    'PhysicianOfRecordIDSequence',
    'PerformingPhysiciansName',
    'PerformingPhysiciansIDSequence',
    'NameOfPhysicianReadingStudy',
    'PhysicianReadingStudyIDSequence',
    'OperatorsName',
    'PatientsName',
    'PatientID',
    'IssuerOfPatientID',
    'PatientsBirthDate',
    'PatientsBirthTime',
    'PatientsSex',
    'OtherPatientIDs',
    'OtherPatientNames',
    'PatientsBirthName',
    'PatientAge',
    'PatientsAddress',
    'PatientsMothersBirthName',
    'CountryOfResidence',
    'RegionOfResidence',
    'PatientsTelephoneNumbers',
    'StudyID',
    'CurrentPatientLocation',
    'PatientsInstitutionResidence',
    'DateTime',
    'Date',
    'Time',
    'PersonName'
]

deidentification_tag.append(0x00080018)
deidentification_name.append('SOPInstanceUID')

deidentification_name.extend(['PseudoPatientID', 'PseudoStudyDate', 'PseudoStudyTime'])

In [460]:
# input_dir, output_dir -> parent folder
# src_dir, target_dir -> subfolder
# src_path, target_path -> file path
def deidentify(input_dir, output_dir, error_dir, csv_file):
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    if not os.path.exists(error_dir): os.makedirs(error_dir)

    f = open(csv_file, 'w', encoding='utf-8', newline='')
    wr = csv.writer(f)
    wr.writerow(deidentification_name)
    deidentified_element_list = []
    
    i = 0

    for path, dirs, files in os.walk(input_dir):
        for file_name in files:
            src_file = file_name
            src_dir = path
            src_path = os.path.join(src_dir, src_file)
            print(i, src_path)
            
            ds = pydicom.read_file(src_path)

            try:
                if len(ds[0x00080005].value) == 2:
                    ds[0x00080005].value = 'ISO_IR 100'
            except KeyError:
                pass

            # ValueError: invalid literal for int() with base 10: 'SD831026'
            # ValueError: invalid literal for int() with base 10: '26535159\x00'
            try:
                pseudo_patient_id = gocept.pseudonymize.integer(int(ds[0x00100020].value.replace('\x00', '')), 'ARw SGfKiy KJN')
            except ValueError:
                pseudo_patient_id = gocept.pseudonymize.text(ds[0x00100020].value, 'ARw SGfKiy KJN')
            # ValueError: embedded null character
            pseudo_patient_name = gocept.pseudonymize.text(__builtins__.str(ds[0x00100010].value).replace('\x00', ''), 'ARw SGfKiy KJN')
            pseudo_study_date = gocept.pseudonymize.date\
            (datetime.strptime(ds[0x00080020].value[0:8], '%Y%m%d').date(), 'ARw SGfKiy KJN').strftime('%Y%m%d')
            pseudo_study_time = gocept.pseudonymize.time\
            (datetime.strptime(ds[0x00080030].value[0:6], '%H%M%S').time(), 'ARw SGfKiy KJN').strftime('%H%M%S')

            for tag in deidentification_tag:
                try:
                    deidentified_element_list.append(ds[tag].value)
                    if tag == 0x00100020: ds[tag].value = __builtins__.str(pseudo_patient_id)
                    elif tag == 0x00100010: ds[tag].value = pseudo_patient_name
                    elif tag == 0x00080020: ds[tag].value = pseudo_study_date
                    elif tag == 0x00080030: ds[tag].value = pseudo_study_time
                    elif tag != 0x00080018: ds[tag].value = ''
                except KeyError:
                    deidentified_element_list.append('')
            ds.remove_private_tags()

            target_file = gocept.pseudonymize.name(os.path.splitext(src_file)[0], 'ARw SGfKiy KJN') + '.dcm'
            target_dir = output_dir + src_dir[len(input_dir):]
            target_path = os.path.join(target_dir, target_file)

            if not os.path.exists(target_dir): os.makedirs(target_dir)

            try:
                pydicom.write_file(target_path, ds)
            except (KeyError, IndexError, TypeError, ValueError) as e:
                print(src_path)
                print(e)

                error_path = os.path.join(error_dir, src_file)
                log_path = os.path.splitext(error_path)[0] + '.txt'
                with open(log_path, 'w') as f:
                    log = src_path + '\r\n' + __builtins__.str(e)
                    f.write(log)
                shutil.move(src_path, error_path)
            else:
                deidentified_element_list.extend([pseudo_patient_id, pseudo_study_date, pseudo_study_time])
                wr.writerow(deidentified_element_list)

            deidentified_element_list = []
            i += 1

    f.close()

In [461]:
csv_file = '/sdb1/share/sacroiliitis/dicom_deidentified/sacroiliitis.csv'
input_dir = '/sdb1/share/sacroiliitis/dicom'
output_dir = '/sdb1/share/sacroiliitis/dicom_deidentified'
error_dir = '/sdb1/share/sacroiliitis/dicom_deidentified/error'

deidentify(input_dir, output_dir, error_dir, csv_file)

0 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.2.392.200036.9125.3.1.2001050510105050001.1280_0000_003059_150890580104a3.dcm
1 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.5.1.963333652324.1268361400281.1_0001_000001_1508903018017c.dcm
2 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20100520.172846.266280_0001_000001_1508902919010a.dcm
3 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.2.392.200046.100.2.1.97032219669.110421122412.8.1_0001_000001_150890243600b7.dcm
4 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.5.1.963333652324.1326682072828.1_0001_000001_150890000401c6.dcm
5 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20120502.131813.8187_0001_000001_15088998960170.dcm
6 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.6.1.963333652324.1441164907453.1_0001_000001_1508896809009a.dcm
7 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46

61 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.5.1.963333652324.1277199760828.1_0001_000001_150890288400ef.dcm
62 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20150504.100409.143643_0001_000001_15088970110132.dcm
63 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.6.1.963333652324.1405395887406.1_0001_000001_1508897693012c.dcm
64 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20150728.114150.170676_0001_000001_150889684900b3.dcm
65 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.5.1.963333652324.1343782500406.1_0001_000001_150889972300c4.dcm
66 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.36.0.1.72567767433727.1487561132974.1_0001_000001_150889543901fb.dcm
67 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20081127.164011.79128_0001_000001_15089040360013.dcm
68 /sdb1/share/sacroiliitis/dicom/train/positive

122 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20150112.153258.323889_0001_000001_150889724001f4.dcm
123 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20160908.140946.58498_0002_000001_15088960640152.dcm
124 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.6.1.963333652324.1377572659109.1_0001_000001_1508898374008e.dcm
125 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20101208.130350.133150_0002_000001_1508902665001b.dcm
126 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20160610.133455.291504_0001_000001_1508896275023b.dcm
127 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.6.1.963333652324.1403595372578.1_0001_000001_1508897663010c.dcm
128 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20160525.114201.286228_0001_000001_1508896295024a.dcm
129 /sdb1/share/sacroiliitis/dicom/train/positive_

183 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.2.392.200036.9125.3.1.2001062710106270003.1280_0000_001152_1508905783047a.dcm
184 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.6.1.963333652324.1480992529546.1_0005_000001_1508895773002f.dcm
185 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20101115.104336.125546_0001_000001_15089027060048.dcm
186 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.30.1.6.1.963333652324.1369800434265.1_0001_000001_1508898532012a.dcm
187 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20120213.161045.455226_0001_000001_150889877401e7.dcm
188 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20141108.120236.672126_0005_000001_15088979150204.dcm
189 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20110124.92811.147143_0001_000001_150890249900f0.dcm
190 /sdb1/share/sacroiliitis/dicom/train/positive_

244 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.2.392.200036.9125.3.1.2005061710506170007.1280_0000_001146_150890540501be.dcm
245 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20090324.104718.97596_0001_000001_150890374200d3.dcm
246 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20080927.94013.60152_0001_000001_15089040790038.dcm
247 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.2.392.200046.100.2.1.44532227491.110616110411.2.1_0001_000001_150889590800b4.dcm
248 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20161110.110743.347266_0001_000001_15088958520080.dcm
249 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802013.2.20161001.104954.333844_0006_000001_150889596400e0.dcm
250 /sdb1/share/sacroiliitis/dicom/train/positive_both/1.3.46.670589.26.802146.2.20141013.90359.287150_0001_000001_150889783501ac.dcm
251 /sdb1/share/sacroiliitis/dicom/train/positive_both/1

...

7066 /sdb1/share/sacroiliitis/dicom/test/negative/1.3.46.670589.26.802146.4.20110901.121821.216341.0.dcm
7067 /sdb1/share/sacroiliitis/dicom/test/negative/1.3.46.670589.26.802146.4.20090107.111317.72558.0.dcm
7068 /sdb1/share/sacroiliitis/dicom/test/negative/1.3.46.670589.30.1.6.1.963333652324.1421199203968.2.dcm
7069 /sdb1/share/sacroiliitis/dicom/test/negative/1.3.46.670589.30.1.6.1.963333652324.1495432314531.2.dcm
7070 /sdb1/share/sacroiliitis/dicom/test/negative/1.3.46.670589.26.802146.4.20161017.104655.71041.0.dcm
7071 /sdb1/share/sacroiliitis/dicom/test/negative/1.3.46.670589.30.1.5.1.963333652324.1324866633843.2.dcm
7072 /sdb1/share/sacroiliitis/dicom/test/negative/1.3.46.670589.30.1.6.1.963333652324.1456102326218.2.dcm
7073 /sdb1/share/sacroiliitis/dicom/test/negative/1.3.46.670589.26.802146.4.20090306.130804.91723.0.dcm
7074 /sdb1/share/sacroiliitis/dicom/test/negative/1.2.840.113619.2.203.4.2147483647.1348705176.716869.dcm
7075 /sdb1/share/sacroiliitis/dicom/test/negative/1.3

7142 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.30.1.6.1.963333652324.1376977613031.1_0002_000001_15088983790092.dcm
7143 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.26.802146.2.20121031.110145.58350_0001_000001_15088996170067.dcm
7144 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.26.802146.2.20140306.134149.216952_0001_000001_15088974870076.dcm
7145 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.30.1.6.1.963333652324.1457320610031.1_0001_000001_150889642302b8.dcm
7146 /sdb1/share/sacroiliitis/dicom/validation/positive/1.2.410.200049.2.3.100.20000123.2.3.20130603152101529_0003_000001_1508898519011b.dcm
7147 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.30.1.5.1.963334633222.1337062890546.1_0001_000001_1508899859014b.dcm
7148 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.30.1.6.1.963333652324.1448351455093.1_0001_000001_15088966670028.dcm
7149 /sdb1/share/sacroiliitis/di

7202 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.26.802013.2.20090630.125815.157666_0001_000001_15089036760098.dcm
7203 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.26.802013.2.20141104.144811.670537_0001_000001_150889790901fd.dcm
7204 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.26.802013.2.20080405.112523.8565_0001_000001_150890421200ba.dcm
7205 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.26.802013.2.20110811.113640.403494_0001_000001_1508902357006f.dcm
7206 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.26.802146.2.20090731.110042.142477_0001_000001_15089036440078.dcm
7207 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.26.802013.2.20090925.93109.191554_0001_000001_15089035940045.dcm
7208 /sdb1/share/sacroiliitis/dicom/validation/positive/1.3.46.670589.30.1.6.1.963333652324.1423808491296.1_0001_000001_150889716501bb.dcm
7209 /sdb1/share/sacroiliitis/dicom/validation/p

7261 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802013.2.20130725.124827.514890_0001_000001_150889131200a3.dcm
7262 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802146.2.20131030.102430.174248_0001_000001_15088911970047.dcm
7263 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.30.1.6.1.963333652324.1367803952296.1_0001_000001_1508891458010d.dcm
7264 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.30.1.6.1.963333652324.1373260137578.1_0002_000001_150889133100b1.dcm
7265 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.30.1.5.1.963333652324.1279153548875.1_0001_000001_150889392601f1.dcm
7266 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802013.2.20121203.101616.72719_0001_000001_1508892775019c.dcm
7267 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802146.2.20100111.134010.41264_0001_000001_1508894157029a.dcm
7268 /sdb1/share/sacroiliitis/dicom/valid

7321 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802146.2.20121108.123222.61150_0001_000001_1508892724017d.dcm
7322 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.30.36.0.1.72567767433727.1453877343112.1_0001_000001_1508889542016d.dcm
7323 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802146.2.20110623.110011.192663_0001_000001_150889335300c1.dcm
7324 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802013.2.20170112.101826.367353_0001_000001_15088593640159.dcm
7325 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802146.2.20170525.104846.149516_0001_000001_150885914700b3.dcm
7326 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802146.2.20170208.153635.111746_0001_000001_15088593190135.dcm
7327 /sdb1/share/sacroiliitis/dicom/validation/negative/1.3.46.670589.26.802013.2.20101202.93917.326862_0001_000001_15088937840182.dcm
7328 /sdb1/share/sacroiliitis/dicom/validati

In [ ]:
csv_file = '/sdb1/share/sacroiliitis/dicom_deidentified/sacroiliitis.csv'
input_dir = '/sdb1/share/sacroiliitis/dicom'
output_dir = '/sdb1/share/sacroiliitis/dicom_deidentified'
error_dir = '/sdb1/share/sacroiliitis/dicom_deidentified/error'

deidentify(input_dir, output_dir, error_dir, csv_file)

In [413]:
csv_file = '/sdb1/share/opll/naesa_positive.csv'
src_dir = '/sdb1/share/opll/positive'
deid_dir = '/sdb1/share/opll/positive_deid'
error_dir = '/sdb1/share/opll/error'

deidentify(src_dir, deid_dir, error_dir, csv_file)

/sdb1/share/opll/positive/2017082526476.1_0001_000002_15102653860025.dcm.dcm
With tag (0028, 3000) got exception: With tag (0028, 3006) got exception: index out of range
/sdb1/share/opll/positive/1.2.410.200033.254693.20170102102707.1_0001_000002_151026566000f8.dcm.dcm
"Unknown DICOM tag (0020, 4001) can't look up VR"
/sdb1/share/opll/positive/1.2.410.200033.254692.20170102102318.1_0001_000004_151026566400fc.dcm.dcm
"Unknown DICOM tag (0020, 4001) can't look up VR"
/sdb1/share/opll/positive/1.2.410.200033.254693.20170102102707.1_0001_000001_151026565800f7.dcm.dcm
"Unknown DICOM tag (0020, 4001) can't look up VR"
/sdb1/share/opll/positive/2017082526476.1_0001_000003_15102653870026.dcm.dcm
With tag (0028, 3000) got exception: With tag (0028, 3006) got exception: index out of range
/sdb1/share/opll/positive/2017082526476.1_0001_000004_15102653880027.dcm.dcm.dcm
With tag (0028, 3000) got exception: With tag (0028, 3006) got exception: index out of range
